## Problem 26 - Reciprocal cycles

In [1]:
import numpy as np

def getLenCycle(d):
    
    r = 1
    rest = []
    exact = False
    
    while True:
        # compute reminders of 1/d division as integer/Euclidean divisions
        r = ( r % d )
        if r == 0: # exact division
            #print("Exact division, no cycle")
            exact = True
            break
        # if reminder already found, cycle found
        if r in rest:
            rest.append(r) # append the repeating reminder, use later to compute cycle lenght
            break
        rest.append(r)
        # otherwise pass to next decimal
        r = r * 10

    lencycle = -1
    
    if not exact:
        rest = np.array(rest)
        rep = rest[-1]
        rec = np.where(rest == rep)
        lencycle = rec[0][1]-rec[0][0]
        
    return lencycle

print("Testing method...")
for d in range(1,10):
    c = getLenCycle(d)
    if (c!=-1):
        print("1 /",d,"=",1/d,"-> cycle =",c)

print("Full execution...")
cmax = 0
dmax = 0 
for d in range(1,1000):
    c = getLenCycle(d)
    #print(d,c,cmax)
    if c > cmax:
        cmax = c
        dmax = d

print("d_max =",dmax,"( cycle lengh =",cmax,")")

Testing method...
1 / 3 = 0.3333333333333333 -> cycle = 1
1 / 6 = 0.16666666666666666 -> cycle = 1
1 / 7 = 0.14285714285714285 -> cycle = 6
1 / 9 = 0.1111111111111111 -> cycle = 1
Full execution...
d_max = 983 ( cycle lengh = 982 )


## Problem 27 - Quadratic primes

In [2]:
from ProjectEuler import primeFactors

def quadraticForm(a,b,n):
    return n*n+a*n+b

def isPrime(n):
    return len(primeFactors(n))==1

#print(isPrime(1), isPrime(51))

lenmax = 0
for a in range(-999,1000):
    for b in range(-1000,1001):
        primes = []
        for n in range(1,max(abs(a),abs(b))):
            p = quadraticForm(a,b,n)
            if isPrime(p):
                primes.append(p)
            else:
                break
        if len(primes)>0 and len(primes)>lenmax:
            lenmax = len(primes)
            print(a,b,len(primes),a*b)

-999 1000 1 -999000
-500 998 2 -499000
-333 993 3 -330669
-247 989 4 -244283
-197 983 5 -193651
-163 983 6 -160229
-131 941 7 -123271
-121 947 8 -114587
-105 967 10 -101535
-61 971 70 -59231


## Problem 28 - Number spiral diagonals

No need to fill all spyral, there's a clear pattern in the diagonal values!

In [3]:
maxside = 1001

sumdiag = 1
n = 1
side = 0 
increment = 2 

while True:

    side += 2 # square side
    # elements on diagonals of a given spyral layer
    for _ in range(4):
        n += increment
        sumdiag += n
        #print(side, n, sumdiag)

    increment += 2 # new spyral layer
    if side==maxside:
        break
        
print(side,sumdiag)

1001 669171001


## Problem 29 - Distinct powers

In [4]:
from collections import defaultdict

count = defaultdict(lambda:0)

amin = 2
#amax = 5
amax = 100

bmin = 2
#bmax = 5
bmax = 100

for a in range(amin,amax+1):
    for b in range(bmin,bmax+1):
        count[a**b] += 1

print(len(count.keys()))

9183


## Problem 30 - Digit fifth powers

In [5]:
def getDigits(n):
    digits = []
    for i in range(len(str(n))-1,-1,-1):
        d = n // 10**i
        digits.append(d)
        n = n - d*10**i
    return digits

def isSumPowDigits(n,p):
    s = 0
    # begin from larger digit in order to stop sum if pows exceeed n faster
    for d in sorted(getDigits(n))[::-1]: 
        s += d**p
        if s > n:
            return False
    if s==n:
        return True
    return False

p = 5

nmax = 1000000
s = 0
for n in range(1,nmax):
    if n!=1 and isSumPowDigits(n,p):
        s += n
        print(n)

print("Sum (excluding 1) =",s)

4150
4151
54748
92727
93084
194979
Sum (excluding 1) = 443839


## Problem 31 - Coin sums

Solution can be found by summing solutions that do not contain m-th coin and solutions that contain at least one m-th coin of `Sm` value. 

I define define a recursive function:

`countCoins(S[], m, n)`

returning as sum of `countCoins(S[], m-1, n)` + `countCoins(S[], m, n-Sm)`, where `Sm` is the value of the m-th coin. The function will need a few "bottom" returns returning 1 for when single coing value is reached, and 0 for when the single coin value is smaller then the desided value.

In [6]:
coinvalues = [1,2,5,10,20,50,100,200]

def countCoins(S, m, n): 
    '''
    Returns count of ways we can sum S[0...m-1] coins to get sum n 
    S = list of coin values
    m = how many coin values to use
    n = total value to obtain
    '''
    
    # If n is 0 then there is 1 solution (do not include any coin) 
    if n == 0: 
        return 1
  
    # If n is less than 0 then no solution exists 
    if n < 0: 
        return 0
    
    # If there are no coins available anymore and n is still greater than 0, then no solution exists
    if m<=0 and n>=0: 
        return 0
  
    # Recursive solution:
    # Sum of solutions including S[m-1] and solutions excluding S[m-1]
    return countCoins(S, m-1, n) + countCoins(S, m, n-S[m-1])

print(countCoins(coinvalues, len(coinvalues), 200)) 

73682


## Problem 32 - Pandigital products

In [7]:
from itertools import permutations
from copy import deepcopy
from collections import defaultdict

digits = list(range(1,10))

products = defaultdict(lambda:0)

# all possible lenghts of first number
for l1 in range(1,len(digits)-2):

    for p1 in permutations(digits,l1):

        # generate first number of lenght l1
        n1 = ""
        for d in p1:
            n1+=str(d)
        n1 = int(n1)

        # generate list of remaining digits
        newdigits = deepcopy(digits)
        for d in p1:
            newdigits.remove(d)

        # all possible lenghts of second number
        for l2 in range(1,len(digits)-l1-1):
       
            for p2 in permutations(newdigits,l2):
                # generate second number of lenght l2 
                n2 = ""
                for d in p2:
                    n2+=str(d)
                n2 = int(n2)

                # generate third number from permutation of remaining digits
                lastdigits = deepcopy(newdigits)
                for d in p2:
                    lastdigits.remove(d)
                
                #print(l1,l2,len(digits)-l1-l2)

                for p3 in permutations(lastdigits,len(digits)-l1-l2):
                    n3 = ""
                    for d in p3:
                        n3+=str(d)
                    n3 = int(n3) 

                    if n1*n2==n3:
                        print(n1,"x",n2,"=",n3)
                        products[n3] +=1
                        
print("Sum of 1-9 pandigital products (ignoring multiple way to obtain one product) =",sum(products.keys())) 

4 x 1738 = 6952
4 x 1963 = 7852
12 x 483 = 5796
18 x 297 = 5346
27 x 198 = 5346
28 x 157 = 4396
39 x 186 = 7254
42 x 138 = 5796
48 x 159 = 7632
138 x 42 = 5796
157 x 28 = 4396
159 x 48 = 7632
186 x 39 = 7254
198 x 27 = 5346
297 x 18 = 5346
483 x 12 = 5796
1738 x 4 = 6952
1963 x 4 = 7852
Sum of 1-9 pandigital products (ignoring multiple way to obtain one product) = 45228


## Problem 33 - Digit cancelling fractions

In [8]:
for den in range(99,11,-1):

    d = den // 10
    u = den - d*10

    if u==0: # skip trivial solutions
        continue
        
    #print(den,d,u)

    # I want to compose all 2-digits integers smalled then den and having at least
    # one digit equal to one of den's digits, so that "simplification" can happen

    for i in range(1,10):
        nums = [ 10*i+d, 10*d+i, 10*i+u, 10*u+i ]
        frac = [ n/den for n in nums ]
        fsim = [ i/u, i/u, i/d, i/d ]
        for j in range(4):
            if frac[j]==fsim[j] and fsim[j]<1.: # avoid fraction greater then 1
                num = int(frac[j]*den)
                ns = i
                if j==0 or j==1:
                    ds = u
                else:
                    ds = d
                #if num // ns != 10 and den // ds != 10: # avoid trivial solution
                print(num,"/",den,"=",ns,"/",ds)

49 / 98 = 4 / 8
19 / 95 = 1 / 5
26 / 65 = 2 / 5
16 / 64 = 1 / 4


> If the product of these four fractions is given in its lowest common terms, find the value of the denominator.

4/8 * 1/5 * 2/5 * 1/4 = 1/100

## Problem 34 

In [9]:
from math import factorial

# start by finding maximum number where sum of factorial of digits is larger or equal of number itself
# to restict the search, here only considering 9! since it gives the largest possible contribution per digit

f9 = factorial(9)
tt = 0
ff = 0
i = 0
while True:
    tt += 9*10**i
    ff += f9
    #print(i,tt,ff)
    if tt>=ff:
        break
    i += 1
    
# ...then (brute force) search!

# Probably not the smartest search: I could have started from the largest number and decreased, 
# and used previous factorial digit sum by applying a correction on the changing digit(s). This 
# anyway assumes that the largest cost is computing factorial(n) for 0-9 digits, but I hope the 
# python implementation I'm using has some kind of memoization: if so, I would not have gained much!

ss = 0

for n in range(1,tt):
    dig = [ int(s) for s in str(n) ] # is there a more clever way to get the digits of a integer?
    s = 0
    for d in sorted(dig)[::-1]: # compute factorial of digits from largest to check for excess and in case break
        s += factorial(d)
        if s > n:
            break
    if s==n and s!=1 and s!=2:
        ss += n
        print(n)
        
print("Result =",ss)

145
40585
Result = 40730


## Problem 35 - Circular primes

In [10]:
# Generates primes numbers list. It needs to contain primes larger than pmax, 
# since *cyclic* permutations (rotations) of digits will be accounted for!
# For 1M it's ok (largest prime < 1M = 999983, which permutations are all smaller)

from ProjectEuler import generatePrimes

primes = []

pmax = 1_000_000

for n in generatePrimes():
    if n>pmax: 
        break
    primes.append(n)
        
print(len(primes),primes[-1])

78498 999983


In [11]:
from itertools import permutations
from collections import defaultdict
import numpy as np

circular = defaultdict(lambda:0)

for p in primes:
    
    digs = [int(s) for s in str(p)]

    skip = False
    for d in digs:
        # if any digit is pair a permutation will be pair, so skip (but keep 2)
        if d in [0,2,4,6,8] and p!=2: 
            skip = True
            break
    if skip:
        continue
    
    perms = []
    permArePrime = True
    
    # generate cyclic permutations (cannot use itertools permutations for this!)
    for i in range(len(digs)):
        ns = ""
        for j in range(i,i+len(digs)):
            ns += str(digs[j%len(digs)])
        n = int(ns)
        if n in primes:
            perms.append(n)
        else:
            permArePrime = False
            break

    if permArePrime:
        for p in perms:
            circular[p] +=1


circ = np.array(sorted(circular.keys()),dtype=int)
print(len(circ[circ<pmax]))

55


## Problem 36 - Double-base palindromes

In [8]:
ssum = 0

for n in range(1,1_000_000):
    b = bin(n)[2:] # remove training '0b'
    s = str(n)
    if b==b[::-1] and s==s[::-1]:
        ssum += n
        print(n,b)
        
print("Sum =",ssum)

1 1
3 11
5 101
7 111
9 1001
33 100001
99 1100011
313 100111001
585 1001001001
717 1011001101
7447 1110100010111
9009 10001100110001
15351 11101111110111
32223 111110111011111
39993 1001110000111001
53235 1100111111110011
53835 1101001001001011
73737 10010000000001001
585585 10001110111101110001
Sum = 872187


## Problem 37 - Truncatable primes

In [65]:
# 1 is not prime. primeFactors(1) returns []

from ProjectEuler import generatePrimes, primeFactors, memoize

# memoize function to check whether number is prime

def isPrime(n):
    f = primeFactors(n)
    if len(f)!=1:
        return False
    else:
        return True

isPrime_mem = memoize(isPrime)

ssum = 0
count = 0

for n in generatePrimes():

    if n in [2, 3, 5, 7]:
        continue
    
    s = str(n)
    truncLeft = True
    for i in range(1,len(s)):
        ss = int(s[i:])
        if not isPrime_mem(ss): # truncation not prime
            truncLeft = False
            break
    
    s = str(n)
    truncRight = True
    for i in range(len(s)-1,0,-1):
        ss = int(s[:i])
        if not isPrime_mem(ss): # truncation not prime
            truncRight = False
            break

    if truncLeft and truncRight:
        count +=1
        ssum += n
        print(count, n)
        
    if count==11:
        break
        
print("Sum of truncatable primes =",ssum)

1 23
2 37
3 53
4 73
5 313
6 317
7 373
8 797
9 3137
10 3797
11 739397
Sum of truncatable primes = 748317


## Problem 38 - Pandigital multiples

In [40]:
def isPandigits(n):
    d = "123456789"
    s = str(n)
    if len(s)!=len(d):
        return False
    for c in d:
        if c not in s:
            return False
    return True    

n = 1
mmax = 1

while True:

    c = ""
    m = 0
    i = 0

    while len(c)<9:
        i += 1
        c += str(n*i)
        m = int(c)
        if isPandigits(m):
            print(n,i,m)
            if m>mmax:
                mmax = m

    if i==1: # not very efficient, but I did not find a more clever condition to stop the loop earlier
        break
        
    n+=1
    
print("Max pandigital multiple =",mmax)

1 9 123456789
9 5 918273645
192 3 192384576
219 3 219438657
273 3 273546819
327 3 327654981
6729 2 672913458
6792 2 679213584
6927 2 692713854
7269 2 726914538
7293 2 729314586
7329 2 732914658
7692 2 769215384
7923 2 792315846
7932 2 793215864
9267 2 926718534
9273 2 927318546
9327 2 932718654
Max pandigital multiple = 932718654


## Problem 39 - Integer right triangles

In [1]:
# Generate Pythagorean Triple accoding to
# https://mathworld.wolfram.com/PythagoreanTriple.html

import numpy as np

def gen_prim_pyth_trips(limit=None):
    u = np.mat(' 1  2  2; -2 -1 -2; 2 2 3')
    a = np.mat(' 1  2  2;  2  1  2; 2 2 3')
    d = np.mat('-1 -2 -2;  2  1  2; 2 2 3')
    uad = np.array([u, a, d])
    m = np.array([3, 4, 5])
    while m.size:
        m = m.reshape(-1, 3)
        if limit:
            m = m[m[:, 2] <= limit]
        yield from m
        m = np.dot(m, uad)

def gen_all_pyth_trips(limit):
    for prim in gen_prim_pyth_trips(limit):
        i = prim
        for _ in range(limit//prim[2]):
            yield i
            i = i + prim

#p = 120
#for a,b,c in gen_all_pyth_trips(p//2):
#    if a+b+c==p:
#        print(a,b,c)
       
from collections import defaultdict

#triplets = defaultdict(lambda: [])
triplets_count = defaultdict(lambda: 0)

p = 1000
for a,b,c in gen_all_pyth_trips(p//2):
    if a+b+c <= p:
        #triplets[a+b+c].append((a,b,c))
        triplets_count[a+b+c]+=1

print(max(triplets_count, key=triplets_count.get))

840


## Problem 40 - Champernowne's constant

In [13]:
d = ""
for i in range(1000000):
    d += str(i)
r = int(d[1]) * int(d[10]) * int(d[100]) * int(d[1000]) * int(d[10000]) * int(d[100000]) * int(d[1000000])
print(r)    

210


## Problem 41 - Pandigital prime

In [15]:
def isPandigits(n):
    s = str(n)
    d = ""
    for i in range(1,len(s)+1):
        d+=str(i)
    if len(s)!=len(d):
        return False
    for c in d:
        if c not in s:
            return False
    return True

from ProjectEuler import generatePrimes

pmax = 987654321

p = 1
for n in generatePrimes():
    if n>=pmax:
        break
    if n>p and isPandigits(n):
        p=n

print("Largest pandigital prime =",p)

Largest pandigital prime = 7652413


## Problem 42 - Coded triangle numbers

In [2]:
# using ASCII codes
alphabet = [ chr(i) for i in range(65,91) ]

def wordValue(w):
    v = 0
    for c in w:
        v+=alphabet.index(c)+1
    return(v)

print(wordValue('SKY'))

55


In [3]:
f = open("./data/p042_words.txt")
values = []
for w in f.read().split(','):
    tw = wordValue(w.replace('"',''))
    values.append(tw)
print("{} words, max value {}".format(len(values),max(values)))

1786 words, max value 192


In [9]:
# Triangle numbers
# tn = ½n(n+1)
# generate only up to maximum word value

triangles = []
for n in range(1,100):
    t = (n*(n+1))//2
    if t<=max(values):
        triangles.append(t)
    else:
        break

triangles

[1, 3, 6, 10, 15, 21, 28, 36, 45, 55, 66, 78, 91, 105, 120, 136, 153, 171, 190]

In [10]:
count = 0
for v in values:
    if v in triangles:
        count += 1
print(count)

162


## Problem 43 - Sub-string divisibility

In [46]:
def isSubstringDivisible(m):
    div = [2,3,5,7,11,13,17]
    for i in range(1,1+len(div)):
        m = int(n[i:i+3])
        if m%div[i-1] != 0:
            return False
    return True

from itertools import permutations

dig = ['0','1','2','3','4','5','6','7','8','9']

s = 0
for d in permutations(dig,10):
    if d[0]=='0':
        continue
    n = "".join(d)
    if isSubstringDivisible(m):
        print(n)
        s += int(n)

print("Sum =",s)

1406357289
1430952867
1460357289
4106357289
4130952867
4160357289
Sum = 16695334890


## Problem 44 - Pentagon numbers

$P_n = n(3n−1)/2$

In [37]:
from collections import defaultdict
from itertools import combinations

Pn = defaultdict(bool)

def pn(n):
    return n*(3*n-1)//2

for n in range(1,10000):
    Pn[pn(n)] = True

for p in combinations(Pn.keys(),2):
    sump = sum(p)
    if Pn[sump]:
        difp = max(p)-min(p)
        if Pn[difp]:
            print(p,sump,difp)
            break

(1560090, 7042750) 8602840 5482660


## Problem 45 - Triangular, pentagonal, and hexagonal

In [58]:
def T(n):
    return n*(n+1)//2

def P(n):
    return n*(3*n-1)//2

def H(n):
    return n*(2*n-1)

from itertools import combinations

Pn = defaultdict(bool)
Tn = defaultdict(bool)
Hn = defaultdict(bool)

for n in range(1,100000):
    Pn[P(n)]=True
    Tn[T(n)]=True
    Hn[H(n)]=True

for p in Pn.keys():
    if Tn[p] and Hn[p] and p>40755:
        print(p)
        break

1533776805


## Problem 46 - Goldbach's other conjecture

In [25]:
from ProjectEuler import primeFactors, generatePrimes
from math import sqrt

def isPrime(n):
    return len(primeFactors(n))==1

def isSquare(n):
    return sqrt(n).is_integer()

def isGoldbach(n):
    #if n==1:
    #    return True
    for p in generatePrimes():
        if p>n:
            return False
        m = (n-p)//2
        if isSquare(m):
            return True

#n = 1
n = 3
while True:
    if not isPrime(n):
        if not isGoldbach(n):
            print(n)
            break
    n += 2

5777


## Problem 47 - Distinct primes factors

In [90]:
from ProjectEuler import primeFactors
from collections import defaultdict, Counter

def isPrime(n):
    return len(primeFactors(n))==1

def primeFactorsPowers(n):
    f = primeFactors(n)
    c = Counter(f)
    return [ i**j for i,j in zip(c.keys(),c.values()) ]

def findConsecutives(k=3):
    factors = defaultdict(lambda: [])
    n = 1
    while True:
        factors[n] = primeFactorsPowers(n)
        if len(factors[n]) != k:
            pass
        else:
            followOK = True
            nextfactors = []
            for i in range(n+1,n+k):
                if not len(factors[i]):
                    factors[i] = primeFactorsPowers(i)
                if len(factors[i])!=k:
                    followOK = False
                    break
                nextfactors = nextfactors + factors[i]
            if not followOK:
                pass
            else:
                select = True
                for f in factors[n]:
                    if f in nextfactors:
                        select = False
                        break
                if select:
                    return [i for i in range(n,n+k)]
        n+=1

In [91]:
print(findConsecutives(2))
print(findConsecutives(3))

[14, 15]
[644, 645, 646]


In [92]:
print(findConsecutives(4))

[134043, 134044, 134045, 134046]


## Problem 48 - Self powers

In [13]:
n = 1000
b = sum([i**i for i in range(1,n+1)])
print(str(b)[-10:])

9110846700


## Problem 49 - Prime permutations

In [20]:
from ProjectEuler import generatePrimes

# 4-digit primes
primes4dig = []
for n in generatePrimes():
    if n<1000:
        pass
    elif 1000<n<9999:
        primes4dig.append(n)
    else:
        break

In [35]:
from itertools import combinations

k = 0
for c in combinations(primes4dig,3):
    p1,p2,p3 = c
    if sorted(str(p1))==sorted(str(p2))==sorted(str(p3)): # are permutations
        if sorted(c)[1]-sorted(c)[0]==sorted(c)[2]-sorted(c)[1]: # have same distance
            print(c,"".join([str(j) for j in sorted(c)]))
            k += 1
    if k==2:
        break

(1487, 4817, 8147) 148748178147
(2969, 6299, 9629) 296962999629


## Problem 50 - Consecutive prime sum

In [75]:
from ProjectEuler import generatePrimes, primeFactors
from collections import defaultdict
def isPrime(n):
    return len(primeFactors(n))==1

def maxPrimeSum(nmax=100,verbose=False):
    
    primedict = defaultdict(bool)
    primes = []
    for n in generatePrimes():
        if n>nmax:
            break
        primes.append(n)
        primedict[n] = True

    pmax = 0
    plen = 0
    ps = 0
    for i in range(2,len(primes)+1):
        if i<plen:
            pass
        
        for j in range(len(primes)-i+1):
            if len(primes[j:j+i])%2==0 and primes[j]!=2:
                pass
            if len(primes[j:j+i])%2!=0 and primes[j]==2:
                pass
            ps = sum(primes[j:j+i])
            if ps>nmax:
                break
            if primedict[ps]:
                if len(primes[j:j+i]) > plen:
                    plen = len(primes[j:j+i])
                    pmax = ps
                    if verbose:
                        print(pmax,plen)
    return pmax

In [78]:
print(maxPrimeSum(100))
print(maxPrimeSum(1000))
print(maxPrimeSum(10000))

41
953
9521


In [79]:
print(maxPrimeSum(100000))

92951


In [82]:
print(maxPrimeSum(1000000))

997651
